In [1]:
import pandas as pd
import numpy as np
import ast

In [29]:
df_study = pd.read_csv('/workspace/data/df_train_study_level_npy640_2.csv', index_col=0)
df_study = df_study.drop('id', axis=1)
df_study["image_id"] = [p.split('/')[-1].split('.')[0] for p in df_study.npy_path]
df_study.head()

,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path,image_id
0,/workspace/data/train/eeecfd50b220/0fd96597c55...,0,1,0,0,3,/workspace/data/train_640_2/13131b0c3db4.png,13131b0c3db4
1,/workspace/data/train/a89c1f3470e1/c23cc53bdbf...,0,0,1,0,4,/workspace/data/train_640_2/238208eb51b4.png,238208eb51b4
2,/workspace/data/train/9cc2f64f0c58/2eebbe88278...,0,1,0,0,2,/workspace/data/train_640_2/fa9729ef16b6.png,fa9729ef16b6
3,/workspace/data/train/7b6c49da06db/c1fd5829f05...,0,1,0,0,2,/workspace/data/train_640_2/b6b631939d4f.png,b6b631939d4f
4,/workspace/data/train/d14080fd6f2a/9d6098d515e...,0,1,0,0,0,/workspace/data/train_640_2/ce289d7a37bb.png,ce289d7a37bb


In [2]:
df = pd.read_csv('/workspace/data/train_image_level.csv')
df.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [43]:
type(row.boxes) == float

True

In [52]:
id_list = []
x = []
y = []
w = []
h = []
study_id = []
image_id = []
have_box = []

for i in range(len(df)):
    row = df.iloc[i]
    # nanの場合でbboxあるかどうか場合分け
    if type(row.boxes) == float:
        id_list += [row.id]
        x += [None]
        y += [None]
        w += [None]
        h += [None]
        study_id += [row.StudyInstanceUID]
        image_id += [row.id.split('_')[0]]
        have_box += [0]
        
    else:
        boxes = ast.literal_eval(row.boxes)
        for dic in boxes:
            id_list += [row.id]
            x += [dic["x"]]
            y += [dic["y"]]
            w += [dic["width"]]
            h += [dic["height"]]
            study_id += [row.StudyInstanceUID]
            image_id += [row.id.split('_')[0]]
            have_box += [1]
        
        
df_bbox = pd.DataFrame({
    "id": id_list,
    "x": x,
    "y": y,
    "w": w,
    "h": h,
    "study_id": study_id,
    "image_id": image_id,
    "have_box": have_box,
})

In [53]:
df_bbox

,id,x,y,w,h,study_id,image_id,have_box
0,000a312787f2_image,789.28836,582.43035,1026.65662,1917.30292,5776db0cec75,000a312787f2,1
1,000a312787f2_image,2245.91208,591.20528,1094.66162,1761.54944,5776db0cec75,000a312787f2,1
2,000c3a3f293f_image,NaN,NaN,NaN,NaN,ff0879eb20ed,000c3a3f293f,0
3,0012ff7358bc_image,677.42216,197.97662,867.79767,999.78214,9d514ce429a7,0012ff7358bc,1
4,0012ff7358bc_image,1792.69064,402.55250,617.02734,1204.35800,9d514ce429a7,0012ff7358bc,1
...,...,...,...,...,...,...,...,...
9888,ffd9b6cf2961_image,707.25199,722.07926,392.14044,849.18683,7eed9af03814,ffd9b6cf2961,1
9889,ffdc682f7680_image,2729.27083,332.26044,1496.25016,2604.58334,a0cb0b96fb3d,ffdc682f7680,1
9890,ffdc682f7680_image,1005.81250,1584.67711,662.22913,775.83337,a0cb0b96fb3d,ffdc682f7680,1
9891,ffe942c8655f_image,208.86463,91.53448,450.96747,628.05473,7d82d53204b8,ffe942c8655f,1


In [54]:
df_bbox.merge(df_study, on="image_id", how="left")

,id,x,y,w,h,study_id,image_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path
0,000a312787f2_image,789.28836,582.43035,1026.65662,1917.30292,5776db0cec75,000a312787f2,1,/workspace/data/train/5776db0cec75/81456c9c542...,0,1,0,0,0,/workspace/data/train_640_2/000a312787f2.png
1,000a312787f2_image,2245.91208,591.20528,1094.66162,1761.54944,5776db0cec75,000a312787f2,1,/workspace/data/train/5776db0cec75/81456c9c542...,0,1,0,0,0,/workspace/data/train_640_2/000a312787f2.png
2,000c3a3f293f_image,NaN,NaN,NaN,NaN,ff0879eb20ed,000c3a3f293f,0,/workspace/data/train/ff0879eb20ed/d8a644cc4f9...,1,0,0,0,0,/workspace/data/train_640_2/000c3a3f293f.png
3,0012ff7358bc_image,677.42216,197.97662,867.79767,999.78214,9d514ce429a7,0012ff7358bc,1,/workspace/data/train/9d514ce429a7/22897cd1daa...,0,1,0,0,1,/workspace/data/train_640_2/0012ff7358bc.png
4,0012ff7358bc_image,1792.69064,402.55250,617.02734,1204.35800,9d514ce429a7,0012ff7358bc,1,/workspace/data/train/9d514ce429a7/22897cd1daa...,0,1,0,0,1,/workspace/data/train_640_2/0012ff7358bc.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9888,ffd9b6cf2961_image,707.25199,722.07926,392.14044,849.18683,7eed9af03814,ffd9b6cf2961,1,/workspace/data/train/7eed9af03814/668a64e3a5f...,0,1,0,0,1,/workspace/data/train_640_2/ffd9b6cf2961.png
9889,ffdc682f7680_image,2729.27083,332.26044,1496.25016,2604.58334,a0cb0b96fb3d,ffdc682f7680,1,/workspace/data/train/a0cb0b96fb3d/ccf363aa080...,0,1,0,0,2,/workspace/data/train_640_2/ffdc682f7680.png
9890,ffdc682f7680_image,1005.81250,1584.67711,662.22913,775.83337,a0cb0b96fb3d,ffdc682f7680,1,/workspace/data/train/a0cb0b96fb3d/ccf363aa080...,0,1,0,0,2,/workspace/data/train_640_2/ffdc682f7680.png
9891,ffe942c8655f_image,208.86463,91.53448,450.96747,628.05473,7d82d53204b8,ffe942c8655f,1,/workspace/data/train/7d82d53204b8/431ee249d16...,0,1,0,0,1,/workspace/data/train_640_2/ffe942c8655f.png


In [55]:
df_bbox.merge(df_study, on="image_id", how="left").to_csv('/workspace/data/df_train_study_level_npy640_2_w_bbox.csv', index=False)

In [56]:
box_df = df_bbox.merge(df_study, on="image_id", how="left")

In [57]:
box_df.image_id

0       000a312787f2
1       000a312787f2
2       000c3a3f293f
3       0012ff7358bc
4       0012ff7358bc
            ...     
9888    ffd9b6cf2961
9889    ffdc682f7680
9890    ffdc682f7680
9891    ffe942c8655f
9892    ffe942c8655f
Name: image_id, Length: 9893, dtype: object